## <font color=orange>Subduction Zone Kinematics: calculating area and volume flux</font>
#### <font color=blue>Notebook 2</font>

This notebook illustrates how the subduction zone kinematics genereated in the first notebook ('Make-Subduction-Stats-Table-notebook1.ipynb') can be used to calculate subducting plate area and slab flux. It also illustrates how this data can be visualised and plotted.

This notebook therefore uses data precomputed and stored in a csv file, and loaded into a pandas data table for manipulation.


This was the second in a series of notebooks used to generate the data presened in the following manuscript:

*'Subduction history reveals Cretaceous slab superflux as a possible cause for the mid-Cretaceous plume pulse and superswell events'* by Madison East, R Dietmar Müller, Simon Williams, Sabin Zahirovic and Christian Heine


(Notebook written by Simon Williams and modified made by Madison East)



In [ ]:
# This cell loads in the required files for the chosen plate model, as well as the precomputed data #
#####################################################################################################

import pygplates
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
from topology_plotting import *
from scipy.special import erfinv
import pandas as pd
import math
from lithosphere_thickness_plate_model import * # this loads in the coded which is used later to convert seafloor age to lithospheric thickness

%matplotlib inline

#######################################################
# Define Input files for Muller 2016 AREPS model
#######################################################
# Specify the name of the GPlates rotation file
rotation_filename = '../Data_AREPS_clean/Global_EarthByte_230-0Ma_GK07_AREPS.rot'

# input topologies to be used with the subduction_convergence script
input_topology_filename = ['../Data_AREPS_clean/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpml',\
             '../Data_AREPS_clean/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpml']

# Static polygons to determine whether subduction segment is adjacent to continent or not
static_polygon_filename = '../Data_AREPS_clean/Shapefiles/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2015_v1.shp'
static_polygon_features = pygplates.FeatureCollection(static_polygon_filename)
continental_polygon_features = []
for feature in static_polygon_features:
    if feature.get_feature_type() == pygplates.FeatureType.gpml_closed_continental_boundary:
        continental_polygon_features.append(feature)


######################################################
rotation_model = pygplates.RotationModel(rotation_filename)

# specify time range and resolution for plots
threshold_sampling_distance_radians = np.radians(0.5)

# Define the time snapshots at which to get the subduction zone properties
min_time = 0.
max_time = 230.
time_step = 1.

# Set the delta time for velocity calculations
velocity_delta_time = 1.

# Typically the achor plate id should be 0
anchor_plate_id = 0

############################################################################################################
# Previously calculated results are loaded here from a .csv file, and stored in a pandas data table.
# This is the data that was generated in the first notebook ('Make-Subduction-Stats-Table-notebook1.ipynb')
df = pd.read_csv('SubductionTable_clean_0_230Ma.csv')

# This takes care of any minor construction errors in the plate model where small sections of convergence zones may momentarily become divergent
# this converts any negative convergence rate to zero (these were very minor occurances in the AREPS model, but if using a new model, check these instances first)
df.ix[df.conv_rate<0,'conv_rate'] = 0

############################################################################################################
# Below we calculate the orthogonal migration and convergence rates, which are important to get true estimates of area and volume flux

mr = np.asarray(df['migr_rate'])
mo = np.asarray(df['migr_obliq'])
df['ortho_migr_rate'] = pd.Series(mr*np.sin(np.radians(np.abs(mo))), index=df.index)

cr = np.asarray(df['conv_rate'])
co = np.asarray(df['conv_obliq'])
df['ortho_conv_rate'] = pd.Series(cr*np.abs(np.cos(np.radians(co))), index=df.index)


### Time for the calculations...

In [ ]:
########### Lithospheric Thickness and Subduction Volume Calculations  ###############

## Thickness (in metres): Equation and constants from 'The Solid Earth' (2005), Sandwell (2001) and Grose et al. (2012) 
## T-values in degrees celcius, thermal diffusivity (k) in m^2/sec
T1 = 1150.
To = 0.
Tm = 1350.
kappa = 0.804e-6
Myr2sec=1e6*365*24*60*60
lithosphere_thickness_boundarylayer = erfinv((T1-To)/(Tm-To))*2*np.sqrt(kappa)*np.sqrt(Myr2sec*np.asarray(df['SeafloorAge']))

for i in range(0, len(lithosphere_thickness_boundarylayer)):
    if (lithosphere_thickness_boundarylayer[i] > 125000) :
        lithosphere_thickness_boundarylayer[i] = 125000
        
## Thickness using translated MATLAB script
lithosphere_thickness_plate_model = list(lithosphere_thickness_boundarylayer)
for i in range(0, len(lithosphere_thickness_boundarylayer)) :
#for i in range(0, 100) :
    lithosphere_thickness_plate_model[i] = plate_isotherm_depth(np.asarray(df['SeafloorAge'])[i], T1)

## To convert arc_length from degrees on a sphere to m (using earth's radius = 6371000 m)
arc_length_m = 2*math.pi*6371000*((np.asarray(df['arc_length']))/360)

## Calculate Subduction Volume (in m^3 per year)
subduction_volume_m3y = (np.asarray(df['ortho_conv_rate'])/100) * lithosphere_thickness_plate_model * arc_length_m

## Calculate Subduction Volume (in m^3 per Ma)
subduction_volume_m3Ma = (np.asarray(df['ortho_conv_rate'])/100) * lithosphere_thickness_plate_model * arc_length_m * 1e6

## Calculate Subduciton Volume (slab flux) (in km^3 per year)
subduction_volume_km3y = subduction_volume_m3y/1e9 

print(len(df))
print(np.asarray(df['SeafloorAge']))
print(lithosphere_thickness_boundarylayer)
print(arc_length_m)
print(subduction_volume_m3y)
#print(lithosphere_thickness_plate_model)
#print(pd.Series(lithosphere_thickness))

# Add the parameters calculated above to our data table
df['arc_length_m'] = pd.Series(arc_length_m, index=df.index)
df['lithosphere_thickness_boundarylayer'] = pd.Series(lithosphere_thickness_boundarylayer, index=df.index)
df['lithosphere_thickness_plate_model'] = pd.Series(lithosphere_thickness_plate_model, index=df.index)
df['subduction_volume_m3y'] = pd.Series(subduction_volume_m3y, index=df.index)
df['subduction_volume_m3Ma'] = pd.Series(subduction_volume_m3Ma, index=df.index)
df['subduction_volume_km3y'] = pd.Series(subduction_volume_km3y, index=df.index)


########## Area calculations ##############

#To calculate subducting plate area (in m^2/year)
subduction_area_m2y = (np.asarray(df['ortho_conv_rate'])/100) * arc_length_m 

# To calculate subducitng plate area (in km^2/year)
subduction_area_km2y = subduction_area_m2y/1e6

df['subduction_area_m2y'] = pd.Series(subduction_area_m2y, index=df.index)
df['subduction_area_km2y'] = pd.Series(subduction_area_km2y, index=df.index)


df_AllTimes = df

# Display the contents of the table containing all subduction
df

In [ ]:
# Save data to csv file!!
df_AllTimes.to_csv('SubductionVolumesAreaTable_clean_%0.0f_%0.0fMa.csv' % (min_time,max_time))

### Visualisation...

The plot below shows time snapshots of the model plate boundaries and allows you to visualise your calcualted data
- each subduction zone segment is represented by a coloured dot, and according to the colour map, depicting what you have chosen to plot (e.g. slab flux or subducting plate area)
- bright blue lines show mid-ocean ridges
- orange lines show 'other' plate boundaries

You can choose what range of time steps you want to plot and what calculation to show. Plot features can be altered and the maps can be exported. By exporting all time steps, you have the potential to generate an animation.


In [ ]:
# import matplotlib.patches as mpatches

transform = ccrs.PlateCarree()

#Define which timesteps you want to plot
times = np.arange(225,231,1)
for time in times:
    subset = df_AllTimes[(df_AllTimes['time']==time)]

    coastlines_file = '../Data_most_recent/Global_EarthByte_230-0Ma_GK07_AREPS_Coastlines.gpmlz'
    pygplates.reconstruct(coastlines_file, rotation_model, 'tmp.shp', time)

    pygplates.reconstruct(continental_polygon_features, rotation_model, 'tmp2.shp', time)

    fig = plt.figure(figsize=(16,6),dpi=150)
    lon0=0
    projection = ccrs.Mollweide(lon0)
    # projection = ccrs.Orthographic(lon0, 30.0)
    ax_map = fig.add_axes([0,0,0.8,1.0], projection=projection)

    # Plot reconstructed coastlines
    coastlines = gpd.read_file("tmp.shp")
    coastlines.plot(ax=ax_map, facecolor="grey", edgecolor="none", alphah=0.8, zorder=0.5, transform=transform)

    # Plot reconstructed continental shelf
    continental_polygons = gpd.read_file("tmp2.shp")
    continental_polygons.plot(ax=ax_map, facecolor="grey", edgecolor="none", alpha=0.3, zorder=0.25, transform=transform)

    plot_velocities_and_topologies(ax_map,input_topology_filename,rotation_model,time,
                                   delta_time=5,res=10,scale=4000,lon0=lon0)

    # in the below lines you define which of your calculations you want to plot e.g. subduction_volume_km3y
    x, y = np.asarray(subset.lon), np.asarray(subset.lat)
    l1 = ax_map.scatter(x,y,c=subset['subduction_volume_km3y'],s=40,edgecolor='',zorder=5,
                   cmap=plt.cm.Blues,vmin=0,vmax=6.5e-1, transform=transform)

    #plt.title('Subduction Volume Flux (%s Ma)' % time, size=18)
    plt.title('%s Ma' % time, size=38)

    plt.colorbar(l1,extend='max',label='Volume ($km^3$/yr)')
    
    #velocity = mpatches.Patch(color='w', label='Velocity Vectors',alpha=0.4)
    #MOR = mpatches.Patch(color='darkturquoise', label='Mid Ocean Ridge',alpha=0.5)
    #transform = mpatches.Patch(color='darkorange', label='Fracture Zones',alpha=0.6)
    #continents = mpatches.Patch(color='grey', label='Continental Shelf',alpha=0.3)
    #coastlines = mpatches.Patch(color='grey', label='Continents',alpha=0.7)
    
    #plt.legend(handles=[velocity, MOR, transform, continents, coastlines], loc=1)
    

    #plt.savefig('Volume_flux_animation/figures_vol/Subduction_Volume_Flux_%s Ma.jpeg' % time ,bbox_inches='tight', dpi=300)

### Total Volume (slab flux)
To understand temporal changes, rather than spatial variation at a single time step, we will need to combine the data from our numerous subduction segments at each time step. The below cell sums the total subduction volume for each 1 Ma timestep and saves it as a csv file. A separate file is made for volume per year and per Ma.

In [ ]:
#Saving Summed Values for 'm^3 per year'

times = np.arange(0,231,1)
total_sub_volume_m3y = []
for time in times:
    subset1 = df_AllTimes[(df_AllTimes['time']==time)]
    total_sub_volume_m3y.append(math.fsum(subset1.subduction_volume_m3y))    
#print(total_sub_volume_m3y)

data = pd.DataFrame()
data['Time_(Ma)'] = pd.Series(times)
data['Subduction_Volume_Flux_(m^3/year)'] = pd.Series(total_sub_volume_m3y)
data
data.to_csv('Summed_data_plus_stats/Subduction_Volume_Flux_Summed_Data_clean_m3y_%0.0f_%0.0fMa.csv' % (min_time,max_time))

In [ ]:
#Saving Summed Values for 'm^3 per Ma'

times = np.arange(0,231,1)
total_sub_volume_m3Ma = []
for time in times:
    subset1 = df_AllTimes[(df_AllTimes['time']==time)]
    total_sub_volume_m3Ma.append(math.fsum(subset1.subduction_volume_m3Ma))    
#print(total_sub_volume_m3Ma)

data = pd.DataFrame()
data['Time_(Ma)'] = pd.Series(times)
data['Subduction_Volume_Flux_(m^3/Ma)'] = pd.Series(total_sub_volume_m3Ma)
data
data.to_csv('Summed_data_plus_stats/Subduction_Volume_Flux_Summed_Data_clean_m3Ma_%0.0f_%0.0fMa.csv' % (min_time,max_time))

In [ ]:
#Saving Summed Values for 'km^3 per yr'

times = np.arange(0,231,1)
total_sub_volume_km3y = []
for time in times:
    subset1 = df_AllTimes[(df_AllTimes['time']==time)]
    total_sub_volume_km3y.append(math.fsum(subset1.subduction_volume_km3y))    
#print(total_sub_volume_km3y)

data = pd.DataFrame()
data['Time_(Ma)'] = pd.Series(times)
data['Subduction_Volume_Flux_(km^3/year)'] = pd.Series(total_sub_volume_km3y)
data
data.to_csv('Summed_data_plus_stats/Subduction_Volume_Flux_Summed_Data_clean_km3y_%0.0f_%0.0fMa.csv' % (min_time,max_time))

### Total Area (subducting plate area)
Here we do the same thing that was done for volume, but for area values instead.

In [ ]:
#Summing area values for subducting plate area (in m^2/yr)

times = np.arange(0,231,1)
total_sub_area_m2y = []
for time in times:
    subset1 = df_AllTimes[(df_AllTimes['time']==time)]
    total_sub_area_m2y.append(math.fsum(subset1.subduction_area_m2y))
#print(total_sub_area_m2y )

data = pd.DataFrame()
data['Time_(Ma)'] = pd.Series(times)
data['Subduction_Area_Flux_(m^2/year)'] = pd.Series(total_sub_area_m2y)
data
data.to_csv('Summed_data_plus_stats/Subduction_Area_Flux_Summed_Data_clean_m2y_%0.0f_%0.0fMa.csv' % (min_time,max_time))

In [ ]:
#Summing area values for subducting plate area (in km^2/yr)

times = np.arange(0,231,1)
total_sub_area_km2y = []
for time in times:
    subset1 = df_AllTimes[(df_AllTimes['time']==time)]
    total_sub_area_km2y.append(math.fsum(subset1.subduction_area_km2y))
#print(total_sub_area_km2y )

data = pd.DataFrame()
data['Time_(Ma)'] = pd.Series(times)
data['Subduction_Area_Flux_(km^2/year)'] = pd.Series(total_sub_area_km2y)
data
data.to_csv('Summed_data_plus_stats/Subduction_Area_Flux_Summed_Data_clean_km2y_%0.0f_%0.0fMa.csv' % (min_time,max_time))